In [507]:
import numpy as np
import datasets

datasets.downloads = '/home/greg/Downloads/'

In [509]:
decode = lambda p: np.argmax(p, axis = 1) 
accuracy = lambda p, Y: np.mean(decode(p) == decode(Y))

In [508]:
trX, teX, trY, teY = datasets.mnist()

In [534]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(output_dim = 100, input_dim = trX.shape[1], activation = 'relu'))
model.add(Dense(output_dim = trY.shape[1], activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')
model.fit(trX, trY, nb_epoch = 1)
print accuracy(model.predict_proba(teX, verbose = 0), teY)

if False:
    config = model.get_config()
    weights = model.get_weights()

    model = Sequential.from_config(config)
    model.set_weights(weights)

Epoch 1/1
60000/60000 [==============================] - 9s - loss: 0.2650     
0.9559


In [542]:
import tensorflow as tf

epochs = 1
batch_size = 32
learning_rate = 0.1
H = 100

N, D = trX.shape
N, C = trY.shape

def dense(output_dim, x):
    shape = (x.get_shape()[1].value, output_dim)
    w = tf.Variable(tf.random_normal_initializer(0.0, 0.05)(shape))
    b = tf.Variable(tf.zeros(shape[1], dtype = tf.float32))
    return tf.matmul(x, w) + b  

x = tf.placeholder(tf.float32, (None, trX.shape[1]))
y = tf.placeholder(tf.float32)
H1 = tf.nn.relu(dense(H, x))  
H2 = dense(trY.shape[1], H1)
loss = tf.nn.softmax_cross_entropy_with_logits(H2, y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    for _ in xrange(epochs):
        for start in xrange(0, len(trX) - batch_size + 1, batch_size):
            end = start + batch_size            
            feed_dict = {x: trX[start : end], y: trY[start : end]}
            sess.run(optimizer, feed_dict = feed_dict)

    print accuracy(H2.eval({x: teX}), teY)

0.9459


In [569]:
# https://github.com/Newmu/Theano-Tutorials

import theano
from theano import tensor as T
from scipy.special import expit

epochs = 1
batch_size = 32
learning_rate = 0.1
H = 100

N, D = trX.shape
N, C = trY.shape

init_weights = lambda n, d: theano.shared(np.random.randn(n, d) * 0.05)

x = T.fmatrix()
y = T.fmatrix()

W1 = init_weights(D, H)
b1 = theano.shared(np.zeros(H) * 1.0)
W2 = init_weights(H, C)
b2 = theano.shared(np.zeros(C) * 1.0)

H1 = T.nnet.relu(T.dot(x, W1) + b1)
H2 = T.nnet.softmax(T.dot(H1, W2) + b2)

loss = T.mean(T.nnet.categorical_crossentropy(H2, y))
updates = [[p, p - T.grad(loss, p) * learning_rate] for p in [W1, W2]]
optimizer = theano.function([x, y], updates = updates, allow_input_downcast = True)

for _ in xrange(epochs):
    for start in xrange(0, len(trX) - batch_size + 1, batch_size):
        end = start + batch_size
        optimizer(trX[start : end], trY[start : end])
        
W1, b1, W2, b2 = W1.get_value(), b1.get_value(), W2.get_value(), b2.get_value()
print accuracy(softmax(np.maximum(0, teX.dot(W1) + b1).dot(W2) + b2), teY)

if False:
    # you can play with theano to write derivative functions

    x = T.fmatrix()
    y = T.fmatrix()
    loss = T.mean(T.nnet.categorical_crossentropy(T.nnet.softmax(x), y))
    grad_loss = theano.function([x, y], T.grad(loss, x))

    x = [[-3, 3], [-2, 1], [-2, 1]]
    y = [[0.0, 1], [0, 1], [1, 0]]
    grad_loss(x, y)

0.9369


In [571]:
import numpy as np
from softmax import softmax

epochs = 10
batch_size = 32
learning_rate = 0.1
# maybe preprocess X
H = 100
init_weights = lambda fan_in, fan_out: np.random.randn(fan_in, fan_out) * 0.05 # try glorot
R = lambda Ws: 0 # regularization penalty

N, D = trX.shape
N, C = trY.shape

W1 = init_weights(D, H)
b1 = np.zeros(H)
W2 = init_weights(H, C)
b2 = np.zeros(C)

# https://github.com/martinkersner/cs231n/blob/master/assignment1/neural_net.py

for _ in xrange(epochs):
    for start in xrange(0, len(trX) - batch_size + 1, batch_size):
        end = start + batch_size
        X = trX[start : end]
        Y = trY[start : end]
        n = len(X)

        # forward propagate
        N1 = X.dot(W1) + b1
        H1 = np.maximum(0, N1)

        N2 = H1.dot(W2) + b2
        H2 = N2

        sm = softmax(H2)
        # loss = -np.mean(np.log(np.sum(sm * Y, axis = 1)))

        # back propagate and update
        grad_loss = (sm - Y) / n 

        grad_H2 = grad_loss
        grad_W2 = H1.T.dot(grad_H2)
        grad_b2 = np.sum(grad_H2, axis = 0)

        grad_H1 = grad_H2.dot(W2.T)
        grad_N1 = N1
        grad_N1[grad_N1 >= 0] = 1
        grad_N1[grad_N1 < 0] = 0
        grad_H1_N1 = grad_H1 * grad_N1
        grad_W1 = X.T.dot(grad_H1_N1)
        grad_b1 = np.sum(grad_H1_N1, axis = 0)

        W2 -= grad_W2 * learning_rate
        b2 -= grad_b2 * learning_rate
        W1 -= grad_W1 * learning_rate
        b1 -= grad_b1 * learning_rate
        
print accuracy(softmax(np.maximum(0, teX.dot(W1) + b1).dot(W2) + b2), teY)

0.975


In [437]:
# http://localhost:8888/edit/Desktop/assignment1/cs231n/gradient_check.py

def numerical_gradient(f, x):
    h = 0.00001
    grad = np.zeros_like(x)
    it = np.nditer(x, flags = ['multi_index'], op_flags = ['readwrite'])
    
    while not it.finished:
        ix = it.multi_index
        oldval = x[ix]
        x[ix] = oldval + h
        fxph = f(x)
        x[ix] = oldval - h
        fxmh = f(x)
        x[ix] = oldval
        grad[ix] = (fxph - fxmh) / (2 * h)
        it.iternext()
        
    return grad

x = np.array([1, 2])
f = lambda x: np.sum(x)
numerical_gradient(f, x)

array([49999, 49999])